In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile0 = pd.read_pickle("FFF_Train_Decile0.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile0.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile0, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 0')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 1s 178us/step - loss: 0.6113 - acc: 0.6573 - val_loss: 0.5971 - val_acc: 0.6846
Epoch 38/100
3055/3055 [==============================] - 1s 177us/step - loss: 0.6128 - acc: 0.6573 - val_loss: 0.5970 - val_acc: 0.6846
Epoch 39/100
3055/3055 [==============================] - 1s 178us/step - loss: 0.6125 - acc: 0.6573 - val_loss: 0.5970 - val_acc: 0.6846
Epoch 40/100
3055/3055 [==============================] - 1s 180us/step - loss: 0.6136 - acc: 0.6573 - val_loss: 0.5971 - val_acc: 0.6846
Epoch 41/100
3055/3055 [==============================] - 1s 180us/step - loss: 0.6110 - acc: 0.6576 - val_loss: 0.5971 - val_acc: 0.6846
Epoch 42/100
3055/3055 [==============================] - 1s 180us/step - loss: 0.6110 - acc: 0.6570 - val_loss: 0.5971 - val_acc: 0.6846
Epoch 43/100
3055/3055 [==============================] - 1s 177us/step - loss: 0.6112 - acc: 0.6576 - val_loss: 0.5971 - val_acc: 0.6846
Epoch 44/100
3055/3055 [=======================

3055/3055 [==============================] - 1s 221us/step - loss: 0.6105 - acc: 0.6573 - val_loss: 0.5967 - val_acc: 0.6846
Epoch 97/100
3055/3055 [==============================] - 1s 208us/step - loss: 0.6093 - acc: 0.6573 - val_loss: 0.5967 - val_acc: 0.6846
Epoch 98/100
3055/3055 [==============================] - 1s 188us/step - loss: 0.6059 - acc: 0.6573 - val_loss: 0.5965 - val_acc: 0.6846
Epoch 99/100
3055/3055 [==============================] - 1s 188us/step - loss: 0.6112 - acc: 0.6576 - val_loss: 0.5965 - val_acc: 0.6846
Epoch 100/100
3055/3055 [==============================] - 1s 188us/step - loss: 0.6150 - acc: 0.6573 - val_loss: 0.5966 - val_acc: 0.6846
Test accuracy: 0.6845549731978571
Train on 3055 samples, validate on 764 samples
Epoch 1/150
3055/3055 [==============================] - 1s 441us/step - loss: 0.6477 - acc: 0.6537 - val_loss: 0.6028 - val_acc: 0.6846
Epoch 2/150
3055/3055 [==============================] - 1s 179us/step - loss: 0.6136 - acc: 0.6573 - va

3055/3055 [==============================] - 1s 202us/step - loss: 0.6041 - acc: 0.6845 - val_loss: 0.5886 - val_acc: 0.6832
Epoch 54/150
3055/3055 [==============================] - 1s 202us/step - loss: 0.6019 - acc: 0.6854 - val_loss: 0.5887 - val_acc: 0.6832
Epoch 55/150
3055/3055 [==============================] - 1s 190us/step - loss: 0.6022 - acc: 0.6897 - val_loss: 0.5887 - val_acc: 0.6832
Epoch 56/150
3055/3055 [==============================] - 1s 181us/step - loss: 0.6006 - acc: 0.6871 - val_loss: 0.5887 - val_acc: 0.6846
Epoch 57/150
3055/3055 [==============================] - 1s 193us/step - loss: 0.6008 - acc: 0.6838 - val_loss: 0.5887 - val_acc: 0.6846
Epoch 58/150
3055/3055 [==============================] - 1s 195us/step - loss: 0.6009 - acc: 0.6881 - val_loss: 0.5886 - val_acc: 0.6832
Epoch 59/150
3055/3055 [==============================] - 1s 180us/step - loss: 0.5989 - acc: 0.6861 - val_loss: 0.5886 - val_acc: 0.6832
Epoch 60/150
3055/3055 [=======================

3055/3055 [==============================] - 1s 193us/step - loss: 0.6000 - acc: 0.6903 - val_loss: 0.5887 - val_acc: 0.6819
Epoch 113/150
3055/3055 [==============================] - 1s 188us/step - loss: 0.6025 - acc: 0.6858 - val_loss: 0.5886 - val_acc: 0.6832
Epoch 114/150
3055/3055 [==============================] - 1s 180us/step - loss: 0.6013 - acc: 0.6841 - val_loss: 0.5886 - val_acc: 0.6819
Epoch 115/150
3055/3055 [==============================] - 1s 177us/step - loss: 0.5992 - acc: 0.6864 - val_loss: 0.5887 - val_acc: 0.6832
Epoch 116/150
3055/3055 [==============================] - 1s 179us/step - loss: 0.6016 - acc: 0.6874 - val_loss: 0.5886 - val_acc: 0.6819
Epoch 117/150
3055/3055 [==============================] - 1s 201us/step - loss: 0.6020 - acc: 0.6877 - val_loss: 0.5886 - val_acc: 0.6819
Epoch 118/150
3055/3055 [==============================] - 1s 190us/step - loss: 0.6017 - acc: 0.6851 - val_loss: 0.5886 - val_acc: 0.6819
Epoch 119/150
3055/3055 [================

...

Epoch 7/200
3055/3055 [==============================] - 1s 180us/step - loss: 0.6100 - acc: 0.6635 - val_loss: 0.5923 - val_acc: 0.6846
Epoch 8/200
3055/3055 [==============================] - 1s 188us/step - loss: 0.6101 - acc: 0.6645 - val_loss: 0.5944 - val_acc: 0.6846

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 9/200
3055/3055 [==============================] - 1s 182us/step - loss: 0.6095 - acc: 0.6674 - val_loss: 0.5930 - val_acc: 0.6846
Epoch 10/200
3055/3055 [==============================] - 1s 180us/step - loss: 0.6030 - acc: 0.6619 - val_loss: 0.5917 - val_acc: 0.6846
Epoch 11/200
3055/3055 [==============================] - 1s 182us/step - loss: 0.6063 - acc: 0.6642 - val_loss: 0.5921 - val_acc: 0.6846
Epoch 12/200
3055/3055 [==============================] - 1s 179us/step - loss: 0.6033 - acc: 0.6707 - val_loss: 0.5923 - val_acc: 0.6950
Epoch 13/200
3055/3055 [==============================] - 1s 182us/step - loss: 0.6068 - acc:

Epoch 65/200
3055/3055 [==============================] - 1s 179us/step - loss: 0.6035 - acc: 0.6694 - val_loss: 0.5915 - val_acc: 0.6937
Epoch 66/200
3055/3055 [==============================] - 1s 182us/step - loss: 0.6047 - acc: 0.6746 - val_loss: 0.5915 - val_acc: 0.6937
Epoch 67/200
3055/3055 [==============================] - 1s 180us/step - loss: 0.6053 - acc: 0.6756 - val_loss: 0.5914 - val_acc: 0.6937
Epoch 68/200
3055/3055 [==============================] - 1s 183us/step - loss: 0.6065 - acc: 0.6733 - val_loss: 0.5914 - val_acc: 0.6937
Epoch 69/200
3055/3055 [==============================] - 1s 182us/step - loss: 0.6088 - acc: 0.6707 - val_loss: 0.5914 - val_acc: 0.6937
Epoch 70/200
3055/3055 [==============================] - 1s 184us/step - loss: 0.6053 - acc: 0.6746 - val_loss: 0.5914 - val_acc: 0.6937
Epoch 71/200
3055/3055 [==============================] - 1s 182us/step - loss: 0.6060 - acc: 0.6753 - val_loss: 0.5914 - val_acc: 0.6937
Epoch 72/200
3055/3055 [==========

3055/3055 [==============================] - 1s 192us/step - loss: 0.6081 - acc: 0.6753 - val_loss: 0.5912 - val_acc: 0.6963
Epoch 125/200
3055/3055 [==============================] - 1s 187us/step - loss: 0.6049 - acc: 0.6759 - val_loss: 0.5912 - val_acc: 0.6963
Epoch 126/200
3055/3055 [==============================] - 1s 182us/step - loss: 0.6045 - acc: 0.6828 - val_loss: 0.5912 - val_acc: 0.6963
Epoch 127/200
3055/3055 [==============================] - 1s 182us/step - loss: 0.6021 - acc: 0.6773 - val_loss: 0.5912 - val_acc: 0.6963
Epoch 128/200
3055/3055 [==============================] - 1s 182us/step - loss: 0.6074 - acc: 0.6815 - val_loss: 0.5913 - val_acc: 0.6990
Epoch 129/200
3055/3055 [==============================] - 1s 180us/step - loss: 0.6028 - acc: 0.6818 - val_loss: 0.5912 - val_acc: 0.6990
Epoch 130/200
3055/3055 [==============================] - 1s 182us/step - loss: 0.6065 - acc: 0.6704 - val_loss: 0.5912 - val_acc: 0.6976
Epoch 131/200
3055/3055 [================

3055/3055 [==============================] - 1s 198us/step - loss: 0.6043 - acc: 0.6825 - val_loss: 0.5910 - val_acc: 0.7003
Epoch 184/200
3055/3055 [==============================] - 1s 193us/step - loss: 0.6055 - acc: 0.6717 - val_loss: 0.5909 - val_acc: 0.7003
Epoch 185/200
3055/3055 [==============================] - 1s 185us/step - loss: 0.6025 - acc: 0.6799 - val_loss: 0.5910 - val_acc: 0.7003
Epoch 186/200
3055/3055 [==============================] - 1s 188us/step - loss: 0.6052 - acc: 0.6845 - val_loss: 0.5910 - val_acc: 0.7003
Epoch 187/200
3055/3055 [==============================] - 1s 186us/step - loss: 0.6043 - acc: 0.6782 - val_loss: 0.5910 - val_acc: 0.7003
Epoch 188/200
3055/3055 [==============================] - 1s 191us/step - loss: 0.6022 - acc: 0.6789 - val_loss: 0.5909 - val_acc: 0.6990
Epoch 189/200
3055/3055 [==============================] - 1s 188us/step - loss: 0.6035 - acc: 0.6769 - val_loss: 0.5909 - val_acc: 0.7003
Epoch 190/200
3055/3055 [================

3055/3055 [==============================] - 1s 194us/step - loss: 0.6044 - acc: 0.6792 - val_loss: 0.5908 - val_acc: 0.6937
Epoch 42/50
3055/3055 [==============================] - 1s 183us/step - loss: 0.6061 - acc: 0.6789 - val_loss: 0.5908 - val_acc: 0.6937
Epoch 43/50
3055/3055 [==============================] - 1s 189us/step - loss: 0.6047 - acc: 0.6828 - val_loss: 0.5908 - val_acc: 0.6937
Epoch 44/50
3055/3055 [==============================] - 1s 223us/step - loss: 0.6036 - acc: 0.6822 - val_loss: 0.5908 - val_acc: 0.6937
Epoch 45/50
3055/3055 [==============================] - 1s 242us/step - loss: 0.6052 - acc: 0.6795 - val_loss: 0.5908 - val_acc: 0.6937
Epoch 46/50
3055/3055 [==============================] - 1s 210us/step - loss: 0.6033 - acc: 0.6897 - val_loss: 0.5908 - val_acc: 0.6924
Epoch 47/50
3055/3055 [==============================] - 1s 193us/step - loss: 0.6051 - acc: 0.6753 - val_loss: 0.5909 - val_acc: 0.6924
Epoch 48/50
3055/3055 [==============================

In [3]:
0.93484

0.93484

In [ ]:
#0.74